# End To End MLOPS project to predict Dog Breed

## Importing the packages

In [ ]:
!pip install ultralytics
!pip install joblib
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import xml.etree.ElementTree as ET
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 6.6 MB/s eta 0:00:00


## Connecting Kaggle

In [ ]:

uploaded = files.upload()
#
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle config view

Saving kaggle.json to kaggle.json
Configuration values from /root/.kaggle
- username: senaabhishek
- path: None
- proxy: None
- competition: None


In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset
!unzip -q stanford-dogs-dataset.zip -d ./stanford-dogs-dataset

 99% 744M/750M [00:09<00:00, 126MB/s]
100% 750M/750M [00:09<00:00, 83.6MB/s]


## Cropping Dog Dataset

In [ ]:
image_dir = './stanford-dogs-dataset/images/Images/'
breed_list = os.listdir(image_dir)

num_classes = len(breed_list)
print("{} breeds".format(num_classes))

n_total_images = 0
for breed in breed_list:
    n_total_images += len(os.listdir(image_dir+"{}".format(breed)))
print("{} images".format(n_total_images))

120 breeds
20580 images


In [ ]:
dataset_dir = "./stanford-dogs-dataset/"
os.mkdir('data')
for breed in breed_list:
    os.mkdir('data/' + breed)
print('Created {} folders to store cropped images of the different breeds.'.format(len(os.listdir('data'))))

for breed in os.listdir('data'):
    for file in os.listdir(dataset_dir+'annotations/Annotation/{}'.format(breed)):
        img = Image.open(dataset_dir+'images/Images/{}/{}.jpg'.format(breed, file))
        tree = ET.parse(dataset_dir+'annotations/Annotation/{}/{}'.format(breed, file))
        xmin = int(tree.getroot().findall('object')[0].find('bndbox').find('xmin').text)
        xmax = int(tree.getroot().findall('object')[0].find('bndbox').find('xmax').text)
        ymin = int(tree.getroot().findall('object')[0].find('bndbox').find('ymin').text)
        ymax = int(tree.getroot().findall('object')[0].find('bndbox').find('ymax').text)
        img = img.crop((xmin, ymin, xmax, ymax))
        img = img.convert('RGB')
        img = img.resize((224, 224))
        img.save('data/' + breed + '/' + file + '.jpg')

Created 120 folders to store cropped images of the different breeds.


## Splitting into train and test set

In [ ]:
data_dir = "./data/"
img_height, img_width = (224,224)
batch_size = 32

In [ ]:
train_ds,val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "both",
    seed =120,
    label_mode = "categorical",
    image_size = (img_height,img_width),
    batch_size = batch_size
)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Using 4116 files for validation.


In [ ]:
class_names = train_ds.class_names

In [ ]:
class_names

['n02085620-Chihuahua',
 'n02085782-Japanese_spaniel',
 'n02085936-Maltese_dog',
 'n02086079-Pekinese',
 'n02086240-Shih-Tzu',
 'n02086646-Blenheim_spaniel',
 'n02086910-papillon',
 'n02087046-toy_terrier',
 'n02087394-Rhodesian_ridgeback',
 'n02088094-Afghan_hound',
 'n02088238-basset',
 'n02088364-beagle',
 'n02088466-bloodhound',
 'n02088632-bluetick',
 'n02089078-black-and-tan_coonhound',
 'n02089867-Walker_hound',
 'n02089973-English_foxhound',
 'n02090379-redbone',
 'n02090622-borzoi',
 'n02090721-Irish_wolfhound',
 'n02091032-Italian_greyhound',
 'n02091134-whippet',
 'n02091244-Ibizan_hound',
 'n02091467-Norwegian_elkhound',
 'n02091635-otterhound',
 'n02091831-Saluki',
 'n02092002-Scottish_deerhound',
 'n02092339-Weimaraner',
 'n02093256-Staffordshire_bullterrier',
 'n02093428-American_Staffordshire_terrier',
 'n02093647-Bedlington_terrier',
 'n02093754-Border_terrier',
 'n02093859-Kerry_blue_terrier',
 'n02093991-Irish_terrier',
 'n02094114-Norfolk_terrier',
 'n02094258-Norwi

## Building CNN model

In [ ]:
model = Sequential()

uploaded_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top= False,
    weights='imagenet',
    input_shape= (224,224,3),
    pooling="max",
    classes=120,

)

for layer in uploaded_model.layers:
  layer.trainable = False

model.add(uploaded_model)
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(120,activation="softmax"))

24274472/24274472 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Functio  (None, 1280)              5919312   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 120)               30840     
                                                                 
Total params: 6737352 (25.70 MB)
Trainable params: 818040 (3.12 MB)
Non-trainable params: 5919312 (22.58 MB)
_____________

## Training the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001) ,loss ='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
epochs = 24
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs
)

Epoch 1/24
515/515 [==============================] - 58s 79ms/step - loss: 1.2664 - accuracy: 0.6654 - val_loss: 0.7177 - val_accuracy: 0.8003
Epoch 2/24
515/515 [==============================] - 35s 68ms/step - loss: 0.6667 - accuracy: 0.7929 - val_loss: 0.6700 - val_accuracy: 0.8052
Epoch 3/24
515/515 [==============================] - 39s 77ms/step - loss: 0.5573 - accuracy: 0.8214 - val_loss: 0.6204 - val_accuracy: 0.8241
Epoch 4/24
515/515 [==============================] - 36s 70ms/step - loss: 0.4923 - accuracy: 0.8427 - val_loss: 0.5894 - val_accuracy: 0.8389
Epoch 5/24
515/515 [==============================] - 40s 77ms/step - loss: 0.4445 - accuracy: 0.8549 - val_loss: 0.6272 - val_accuracy: 0.8243
Epoch 6/24
515/515 [==============================] - 36s 69ms/step - loss: 0.4078 - accuracy: 0.8636 - val_loss: 0.6621 - val_accuracy: 0.8265
Epoch 7/24
515/515 [==============================] - 36s 70ms/step - loss: 0.3913 - accuracy: 0.8717 - val_loss: 0.6843 - val_accuracy:

In [ ]:
uploaded = files.upload()

Saving Chihuahua_test2.png to Chihuahua_test2.png
Saving Italian_greyhound_test3.png to Italian_greyhound_test3.png


In [ ]:
image = cv2.imread(str(os.path.join('/content/Labrador_retriever_test1.png')))
image_resized = cv2.resize(image,(img_height,img_width))
image = np.expand_dims(image_resized,axis=0)

pred = model.predict(image)
pred_class = class_names[np.argmax(pred)]
print(pred_class[10:])

1/1 [==============================] - 3s 3s/step
Weimaraner


## Model predictions







### we need the input image to be cropped. So we are going to use YOLOv8 to make boundary box around the dog image and crop the dog only.




In [ ]:
from ultralytics import YOLO

model_object_detection = YOLO('yolov8n.pt')
print(model_object_detection.names)

100%|██████████| 6.23M/6.23M [00:00<00:00, 122MB/s]

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [ ]:
image = cv2.imread(str(os.path.join('/content/Chihuahua_test2.png')))
image_resized = cv2.resize(image,(img_height,img_width))
image = np.expand_dims(image_resized,axis=0)

pred = model.predict(image)
pred_class = class_names[np.argmax(pred)]
print(pred_class[10:])

1/1 [==============================] - 3s 3s/step
Chihuahua


In [ ]:
def detect_dog(frame):
  detections = model_object_detection(frame)[0]
  detections_ = []
  for detection in detections.boxes.data.tolist():
    #print(detection)
    x1,y1,x2,y2,score, class_id = detection
    if int(class_id) == 16:
      detections_.append([x1,y1,x2,y2,score])
  return detections_


def crop_only_bb(image_path):
  dog = cv2.imread(str(os.path.join(image_path)))
  (x_center,y_center,bb_height,bb_width,score) = detect_dog(dog)[0]
  image_width = dog.shape[1]
  image_height = dog.shape[0]
  dog_cropped = dog[int(y_center):int(bb_width), int(x_center):int(bb_height),:]
  cv2.imwrite('cropped_img.jpg', dog_cropped)

### Uploading the image and prediction

In [ ]:
uploaded = files.upload()

In [ ]:
crop_only_bb('/content/Chihuahua_test2.png')
image = cv2.imread(str(os.path.join('/content/cropped_img.jpg')))
image_resized = cv2.resize(image,(img_height,img_width))
image = np.expand_dims(image_resized,axis=0)
pred = model.predict(image)
pred_class = class_names[np.argmax(pred)]
print(pred_class[10:])

### Downloading the model created